# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Ключевые задачи проекта:


**1. Общий анализ данных**
    
    - открытие файла
    - общий осмотр информации датасета
    - написание общих выводов по качеству данных


**2. Предобработка данных**
    
    - обработка пропусков
    - коррекция типов данных
    - обработка дубликатов
    - лемматизация
    - категоризация данных


**3. Ответы на вопросы**
    
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?
    
    
**4. Общий вывод по работе**


## Шаг 1. Общий анализ

In [31]:
#импортируем библиотеки, производим общий осмотр данных
import pandas as pd
import numpy as np

df = pd.read_csv('/datasets/data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [32]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [34]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [35]:
df.shape

(21525, 12)

In [36]:
df.columns.to_list()

['children',
 'days_employed',
 'dob_years',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

**Вывод**

**Общие наблюдения и задачи:**

- изменить названия столбцов на более показательный (children -> number_of_children)
- коррекция типов данных в столбцах (дни стажа -> integer)
- дни стажа перевести в года
- для лучшей визуализации перевод total_income в целочисленный формат
- обработка пропущенных значений (days_employed, total_income)
- минимальный возраcт клиента равно нулю
- приведение к единому регистру и удаление дубликатов
- отрицательный значение (-1 в children , days_employed)
- для education и family_status в таблице приведены и значения, и ключи (если используются ключи, то можно выводить значения в отдельную таблицу, чтобы не нагромождать основную). 



## Шаг 2. Предобработка данных

### Обработка пропусков

In [37]:
#Поменяем названия некоторых столбцов
new_columns = ['number_of_children',
 'days_employed',
 'debtor_age',
 'education_type',
 'education_type_id',
 'family_status_type',
 'family_status_type_id',
 'gender',
 'job_type',
 'debt_problems',
 'month_income',
 'purpose_of_credit']

df.set_axis(new_columns, axis = 'columns', inplace = True)
df.columns.to_list()

['number_of_children',
 'days_employed',
 'debtor_age',
 'education_type',
 'education_type_id',
 'family_status_type',
 'family_status_type_id',
 'gender',
 'job_type',
 'debt_problems',
 'month_income',
 'purpose_of_credit']

In [38]:
#Проанализируем пропуски в данных
df.isna().sum()

number_of_children          0
days_employed            2174
debtor_age                  0
education_type              0
education_type_id           0
family_status_type          0
family_status_type_id       0
gender                      0
job_type                    0
debt_problems               0
month_income             2174
purpose_of_credit           0
dtype: int64

2174 пропуска в стаже + 2174 пропуска в доходе

In [39]:
df[df['days_employed'].isna()==True].head(15)

,number_of_children,days_employed,debtor_age,education_type,education_type_id,family_status_type,family_status_type_id,gender,job_type,debt_problems,month_income,purpose_of_credit
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [40]:
df[df['days_employed'].isna()==True]['job_type'].value_counts(normalize=True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: job_type, dtype: float64

In [41]:
df[df['days_employed'].isna()==False]['job_type'].value_counts(normalize=True)

сотрудник          0.517493
компаньон          0.236525
пенсионер          0.177924
госслужащий        0.067800
безработный        0.000103
предприниматель    0.000052
студент            0.000052
в декрете          0.000052
Name: job_type, dtype: float64

In [42]:
df[df['days_employed'].isna()==True]['family_status_type'].value_counts(normalize=True)

женат / замужем          0.568997
гражданский брак         0.203312
Не женат / не замужем    0.132475
в разводе                0.051518
вдовец / вдова           0.043698
Name: family_status_type, dtype: float64

In [43]:
df[df['days_employed'].isna()==False]['family_status_type'].value_counts(normalize=True)

женат / замужем          0.575836
гражданский брак         0.193013
Не женат / не замужем    0.130484
в разводе                0.055966
вдовец / вдова           0.044701
Name: family_status_type, dtype: float64

Не указали свой доход люди с разным типом занятости, возрастов, типом образования, а также разными целями на кредит.
Заменять на ноль не совсем корректно, это снизит шанс выдачи кредита. Пропуски могут быть связаны с отсуствием трудового стажа и официального заработка по определенным причинам (число пропусков совпадает двух столбцов).

**Задача:** 

    - Подготовить данные в столбцах days_employed и month_income для нахождения корректной медианы
    - Найти медиану для разных типов клиентов (по имеющимся группам образования и занятости)
    - Каждому отдельно взятому пропуску присвоить найденную медиану

In [44]:
#подготавливаем данные в столбцах days_employed и month_income
df['days_employed'] = df['days_employed'].apply(abs)
df['month_income'] = df['month_income'].apply(abs)

In [45]:
#Найдем медиану для разных типов клиентов (по имеющимся группам образования и занятости)
df_median = pd.pivot_table(
                            df,
                            index = ['education_type', 'job_type'],
                            values = ['days_employed', 'month_income'],
                            aggfunc = np.median)

df_median  

days_employed   month_income
education_type      job_type                                     
ВЫСШЕЕ              госслужащий        1424.562274  174909.179988
                    компаньон          1423.978136  200701.997023
                    пенсионер        367532.282115  165563.164062
                    сотрудник          1602.753306  144683.271465
Высшее              безработный      395302.838654  202722.511368
                    госслужащий        3750.610464  161671.347625
                    компаньон          1453.451901  192451.667511
                    пенсионер        368379.850772  154962.528710
                    сотрудник          1534.735406  178238.955336
НАЧАЛЬНОЕ           компаньон          1265.943306  190952.908390
                    пенсионер        360264.985350   90479.336389
                    сотрудник          1012.313022  212545.366151
НЕОКОНЧЕННОЕ ВЫСШЕЕ компаньон           775.760267  145528.679914
                    пенсионер        358732.475441  412386.347106
                    сотрудник          1546.148345  166015.790504
Начальное           госслужащий         301.333707  190912.178349
                    компаньон          1030.219648   88227.830722
                    пенсионер        364632.447491   98853.333449
                    сотрудник           852.697747  129895.001992
Неоконченное высшее госслужащий        1178.018163  107224.860090
                    компаньон           711.837088  183379.320368
                    пенсионер        396568.887159   83625.583859
                    сотрудник          1653.837028  159115.726175
СРЕДНЕЕ             в декрете          3296.759962   53829.130729
                    госслужащий        3356.097160  132756.053048
                    компаньон          1945.914306  157308.365600
                    пенсионер        363541.465259  113656.027279
                    сотрудник          1567.112656  135823.934197
Среднее             госслужащий        2291.747052  148529.629601
                    компаньон          1863.730566  149190.998579
                    пенсионер        363995.316389  128155.229215
                    сотрудник          1730.667138  133399.107243
УЧЕНАЯ СТЕПЕНЬ      сотрудник           409.200149  198570.757322
Ученая степень      пенсионер        337584.815560   98752.495442
высшее              госслужащий        2577.299850  172511.107016
                    компаньон          1461.167287  201911.716329
                    пенсионер        366157.236636  142825.046222
                    предприниматель     520.848083  499163.144947
                    сотрудник          1548.646517  166164.078024
                    студент             578.751554   98201.625314
начальное           госслужащий        3043.933615  105766.403301
                    компаньон          1166.810199  170077.891115
                    пенсионер        358777.901745  104505.757848
                    сотрудник          1261.719742  124298.759469
неоконченное высшее госслужащий        2038.132156  165106.669771
                    компаньон          1032.968329  179867.152890
                    пенсионер        371193.915404  122706.716059
                    сотрудник          1158.032596  149551.191876
среднее             безработный      337524.466835   59956.991984
                    госслужащий        2856.781570  136652.970357
                    компаньон          1645.331963  159652.900458
                    пенсионер        365232.442222  114483.373934
                    сотрудник          1607.383724  136818.115423
ученая степень      госслужащий        5968.075884  111392.231107
                    пенсионер        376276.219531  255425.196556
                    сотрудник          3851.735057  192180.127162

In [46]:
#функция для заполнения пропусков по days_employed

def fill_days_employed(row):
    return df_median.loc[row['education_type']].loc[row['job_type']]['days_employed']

In [47]:
#функция для заполнения пропусков по month_income

def fill_month_income(row):
    return df_median.loc[row['education_type']].loc[row['job_type']]['month_income']

In [49]:
df.loc[df['days_employed'].isna() == True, 'days_employed'] = df.apply(fill_days_employed, axis=1)

In [50]:
df.loc[df['month_income'].isna() == True, 'month_income'] = df.apply(fill_month_income, axis=1)

In [51]:
df.isna().sum()

number_of_children       0
days_employed            0
debtor_age               0
education_type           0
education_type_id        0
family_status_type       0
family_status_type_id    0
gender                   0
job_type                 0
debt_problems            0
month_income             0
purpose_of_credit        0
dtype: int64

**Избавились от пропусков. Перепроверили результат. Прежде чем переходить к замене типа данных:** 

1) решим проблему отрицательных значений (number_of_children), 
и 2) рассмотрим клиентов, кому 0 лет.


In [52]:
#решим проблему отрицательных значений (number_of_children)
df['number_of_children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: number_of_children, dtype: int64

Замечаем два артефаект - (-1) и (20). Для -1 предполагаем тире и единица. 

Для 20 предполагаем лишний ноль в конце (так как все значения находятся в интервале 0 - 5, и после резко выпадающее значение 20).

In [53]:
df['number_of_children'] = df['number_of_children'].apply(abs)
df['number_of_children'] = df['number_of_children'].replace(20, 2)

In [54]:
df['number_of_children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: number_of_children, dtype: int64

In [55]:
#рассмотрим клиентов с возрастом 0 лет
df[df['debtor_age']==0]

,number_of_children,days_employed,debtor_age,education_type,education_type_id,family_status_type,family_status_type_id,gender,job_type,debt_problems,month_income,purpose_of_credit
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1607.383724,0,среднее,1,женат / замужем,0,F,сотрудник,0,136818.115423,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


In [56]:
df[df['debtor_age']==0]['debtor_age'].count()

101

101 клиент без данных о возрасте. Не будем удалять данные строки для сохранения корреляции по остальным параметрам.

Добавим в датасет трудовой стаж в годах.

In [57]:
df['years_employed'] = df['days_employed'] / 365
df.describe()

,number_of_children,days_employed,debtor_age,education_type_id,family_status_type_id,debt_problems,month_income,years_employed
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04,21525.000000
mean,0.479721,67302.038726,43.293380,0.817236,0.972544,0.080883,1.653759e+05,184.389147
std,0.755528,139404.982703,12.574584,0.548138,1.420324,0.272661,9.816425e+04,381.931459
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04,0.066141
25%,0.000000,1025.402943,33.000000,1.000000,0.000000,0.000000,1.077142e+05,2.809323
50%,0.000000,1990.714946,42.000000,1.000000,0.000000,0.000000,1.431116e+05,5.454014
75%,1.000000,5347.024506,53.000000,1.000000,1.000000,0.000000,1.981056e+05,14.649382
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06,1100.699727


Замечаем проблемные значения по годам (и по days_employed) трудового стажа. Максимум составляет 1367.6 лет. Оставим эти данные для сохранения связи по другим параметрам, так как этот срез содержит 5619 строчек. 

In [58]:
df[df['years_employed']>100]

,number_of_children,days_employed,debtor_age,education_type,education_type_id,family_status_type,family_status_type_id,gender,job_type,debt_problems,month_income,purpose_of_credit,years_employed
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814
12,0,365232.442222,65,среднее,1,гражданский брак,1,M,пенсионер,0,114483.373934,сыграть свадьбу,1000.636828
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля,1096.660649
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью,927.539597
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости,996.023258
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу,928.506483
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость,1058.897847
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью,992.222066
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем,1024.645783


**Вывод**

Избавились от пропусков, произвели замену пропусков используя корреляцию по имеющимся столбцам. Проанализировали проблемные места в данных.

### Замена типа данных

In [59]:
#заменяем тип данных для лучшей визуализации и репрезентативности
df['days_employed'] = df['days_employed'].astype('int')
df['years_employed'] = df['years_employed'].astype('int')
df['month_income'] = df['month_income'].astype('int')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
number_of_children       21525 non-null int64
days_employed            21525 non-null int64
debtor_age               21525 non-null int64
education_type           21525 non-null object
education_type_id        21525 non-null int64
family_status_type       21525 non-null object
family_status_type_id    21525 non-null int64
gender                   21525 non-null object
job_type                 21525 non-null object
debt_problems            21525 non-null int64
month_income             21525 non-null int64
purpose_of_credit        21525 non-null object
years_employed           21525 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


In [60]:
df.head()

,number_of_children,days_employed,debtor_age,education_type,education_type_id,family_status_type,family_status_type_id,gender,job_type,debt_problems,month_income,purpose_of_credit,years_employed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932


**Вывод**

Преобразовали тип данных для трудового стажа и дохода в целочисленный формат методом .astype. 

### Обработка дубликатов

In [61]:
#проверяем уникальные значения для education_type, family_status_type, job_type, purpose_of_credit для анализа проблем с регистрами.
df['education_type'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [62]:
df['family_status_type'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [63]:
df['job_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [64]:
df['purpose_of_credit'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

До удаления дубликатов необходимо привести буквы колонки education к нижнему регистру.

In [65]:
df['education_type'] = df['education_type'].str.lower()

In [66]:
df.duplicated().sum()

54

In [67]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод**

Привели категориальные данные типов образования к нижнему регистру. Удалили дубликаты. Перепроверили результат. Дубликаты могут возникать вследствии отсутствия системы валидации на приемном сервере.

### Лемматизация

In [68]:
from pymystem3 import Mystem
from collections import Counter

m = Mystem()

In [69]:
credit_reasons = df['purpose_of_credit'].unique()
credit_reasons_string = ', '.join(credit_reasons)
credit_reasons_string

'покупка жилья, приобретение автомобиля, дополнительное образование, сыграть свадьбу, операции с жильем, образование, на проведение свадьбы, покупка жилья для семьи, покупка недвижимости, покупка коммерческой недвижимости, покупка жилой недвижимости, строительство собственной недвижимости, недвижимость, строительство недвижимости, на покупку подержанного автомобиля, на покупку своего автомобиля, операции с коммерческой недвижимостью, строительство жилой недвижимости, жилье, операции со своей недвижимостью, автомобили, заняться образованием, сделка с подержанным автомобилем, получение образования, автомобиль, свадьба, получение дополнительного образования, покупка своего жилья, операции с недвижимостью, получение высшего образования, свой автомобиль, сделка с автомобилем, профильное образование, высшее образование, покупка жилья для сдачи, на покупку автомобиля, ремонт жилью, заняться высшим образованием'

In [70]:
#посмотрим на самые популярные цели кредита среди запросов
top_purposes = []
for purpose in df['purpose_of_credit']:
    lemma = m.lemmatize(purpose)
    for word in lemma:
        if word != " " and word != '\n':
            top_purposes.append(word)
print(Counter(top_purposes))


Counter({'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


In [71]:
# Код ревьюера

stop_words = ['с','для','на','свой',' ', '\n','со']
top_purposes_demo = []
for purpose in df['purpose_of_credit']:
    lemma = ' '.join(filter(lambda x : x not in stop_words,m.lemmatize(purpose)))
    top_purposes_demo.append(lemma)
    
print(Counter(top_purposes_demo))

Counter({'автомобиль': 1451, 'операция недвижимость': 1302, 'покупка жилье': 1266, 'покупка автомобиль': 977, 'свадьба': 793, 'проведение свадьба': 773, 'сыграть свадьба': 769, 'покупка коммерческий недвижимость': 662, 'операция жилье': 652, 'покупка жилье сдача': 652, 'операция коммерческий недвижимость': 650, 'жилье': 646, 'покупка жилье семья': 638, 'строительство собственный недвижимость': 635, 'недвижимость': 633, 'строительство жилой недвижимость': 625, 'покупка недвижимость': 621, 'строительство недвижимость': 619, 'ремонт жилье': 607, 'покупка жилой недвижимость': 606, 'заниматься высокий образование': 496, 'сделка подержанный автомобиль': 486, 'покупка подержать автомобиль': 478, 'приобретение автомобиль': 461, 'дополнительный образование': 460, 'сделка автомобиль': 455, 'высокий образование': 452, 'образование': 447, 'получение дополнительный образование': 447, 'получение образование': 442, 'профильный образование': 436, 'получение высокий образование': 426, 'заниматься образ

**Вывод**

Самые популярные цели кредита: недвижимость, жилье, автомобиль, образование.

### Категоризация данных

Категоризацию проведем с выделение групп для ответа на поставленные вопросы в пункте 3 (по количеству детей, по уровню дохода, по целям кредита). Напишем 3 функции категоризации и используем метод apply.

In [72]:
np.sort(df['number_of_children'].unique())

array([0, 1, 2, 3, 4, 5])

In [73]:
def get_children_group(number_of_children):
    children_groups = {
        0 : 'бездетная семья',
        1 : 'однодетная семья',
        2 : 'двухдетная семья'
    }
    return children_groups.get(number_of_children, 'многодетная семья')

In [74]:
df['month_income'].quantile([0.25, 0.5, 0.75])

0.25    107520.0
0.50    143206.0
0.75    198251.5
Name: month_income, dtype: float64

In [75]:
def get_income_group(income):
    if income < 107520.0:
        return "Низкий доход"
    elif income < 143206.0:
        return "Средний доход"
    elif income < 198251.5:
        return "Высокий доход"
    return "Сверхвысокий доход"

In [76]:
pd.qcut(df['month_income'], 4)

0        (198251.5, 2265604.0]
1         (107520.0, 143206.0]
2         (143206.0, 198251.5]
3        (198251.5, 2265604.0]
4         (143206.0, 198251.5]
                 ...          
21466    (198251.5, 2265604.0]
21467     (143206.0, 198251.5]
21468    (20666.999, 107520.0]
21469    (198251.5, 2265604.0]
21470    (20666.999, 107520.0]
Name: month_income, Length: 21471, dtype: category
Categories (4, interval[float64]): [(20666.999, 107520.0] < (107520.0, 143206.0] < (143206.0, 198251.5] < (198251.5, 2265604.0]]

In [77]:
#напишем функцию для определения цели кредита по найденному топ-списку целей
def get_reason_group(reason):
    if 'недвиж' in reason:
        return "Недвижимость"
    elif 'жиль' in reason:
        return "Жилье"
    elif 'авто' in reason:
        return "Автомобиль"
    elif 'образ' in reason:
        return "Образование"
    return "Остальное"

In [78]:
#создадим новые столбцы на основе написанных функций
df['number_of_children_group'] = df['number_of_children'].apply(get_children_group)
df['month_income_group'] = df['month_income'].apply(get_income_group)
df['credit_purpose_group'] = df['purpose_of_credit'].apply(get_reason_group)
df

,number_of_children,days_employed,debtor_age,education_type,education_type_id,family_status_type,family_status_type_id,gender,job_type,debt_problems,month_income,purpose_of_credit,years_employed,number_of_children_group,month_income_group,credit_purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,однодетная семья,Сверхвысокий доход,Жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,однодетная семья,Средний доход,Автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,бездетная семья,Высокий доход,Жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,многодетная семья,Сверхвысокий доход,Образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,бездетная семья,Высокий доход,Остальное
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12,однодетная семья,Сверхвысокий доход,Жилье
21467,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,942,бездетная семья,Высокий доход,Автомобиль
21468,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5,однодетная семья,Низкий доход,Недвижимость
21469,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8,многодетная семья,Сверхвысокий доход,Автомобиль


**Вывод**

Произвели категоризацию данных. Проверили результат на итоговой таблице.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [79]:
pd.pivot_table(df, index = 'number_of_children_group', values = 'debt_problems')

,debt_problems
number_of_children_group,
бездетная семья,0.075353
двухдетная семья,0.094925
многодетная семья,0.081579
однодетная семья,0.091639


In [80]:
df['number_of_children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: number_of_children, dtype: int64

**Вывод**

Минимальный процент невозврата у бездетных семей, около 7.5%, возможно, из-за отсутствия материальной нагрузки на содержание детей. Однако для многодетных семей этот процент (8.1%) ниже, чем у одно- и двухдетных семей (9.2 и 9.5%, соответственно). Причиной этого может быть начальный этап построения карьеры для одно- двухдетных семей, и сформировавшаяся карьера и более высокий заработок для многодетных семей. Следует отметить, что группа "многодетная семья" состоит из 380 клиентов, что вызывает некоторую неопределенность в плане представительности выборки по сравнению, например, с бездетными семьями (14107 клиентов). Необходимо дополнять датасет в равных пропорциях для лучшей достоверности результатов. Итого, максимальная разница процента невозврата между группами около 2%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [81]:
pd.pivot_table(df, index = 'family_status_type', values = 'debt_problems')

,debt_problems
family_status_type,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093202
женат / замужем,0.075421


In [82]:
df['family_status_type'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status_type, dtype: int64

**Вывод**

Самый большой процент невозврата в группе "не женат / не замужем" (9.8%), а также людей в гражданском браке (9.3%). По остальным трем группам "в разводе" "вдовец/вдова" "женат/замужем" разница невелика (0.5%), а среднее значение составляет 7.1%. Максимальная разница между всеми группами составляет 3.2%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [83]:
pd.pivot_table(df, index = 'month_income_group', values = 'debt_problems')

,debt_problems
month_income_group,
Высокий доход,0.085507
Низкий доход,0.079545
Сверхвысокий доход,0.069672
Средний доход,0.089622


In [84]:
df['month_income_group'].value_counts(normalize=True)

Сверхвысокий доход    0.250012
Высокий доход         0.250012
Низкий доход          0.250012
Средний доход         0.249965
Name: month_income_group, dtype: float64

**Вывод**

Группа людей со сверхвысоким доходом для данного датасета характеризуется минимальным процентом невозврата (7%). Однако, далее видно, что люди со средним и высоким заработком отдают кредит реже (9 и 8.6% по невозврату, соответственно) группы с низким доходом (8%). Максимальная разница - группами со средним и сверхвысоким уровнями дохода - 2%.

- Как разные цели кредита влияют на его возврат в срок?

In [85]:
pd.pivot_table(df, index = 'credit_purpose_group', values = 'debt_problems')

,debt_problems
credit_purpose_group,
Автомобиль,0.093547
Жилье,0.069043
Недвижимость,0.074610
Образование,0.092177
Остальное,0.079657


In [86]:
df['credit_purpose_group'].value_counts()

Недвижимость    6353
Жилье           4461
Автомобиль      4308
Образование     4014
Остальное       2335
Name: credit_purpose_group, dtype: int64

**Вывод**

Давать кредит на жилье и недвижимость выгоднее (процент невозврата 6.9 и 7.5%, соответственно), чем на образование и автомобиль (9.2 и 9.4%, соответственно). Это может быть связано с тем, что жилье - ключевая потребность для клиента, при этом автомобиль - может быть импульсивной покупкой, а в процессе получения образования - сложность заработка в связи с обучением. Максимальная разница по всем группам - 2.5%.

## Шаг 4. Общий вывод

**По исследованию зависимости групп клиентов и процента невозврата:**

*Приблизительная разница по проценту невозврата между группа клиентов в экспериментах - около 2.5%. Учитывая объемы финансовых потоков банка, этот показатель (2.5%) значителен.*

- Исходя из анализа, банку стоит сфокусироваться на выдаче кредитов бездетным семьям (7.1% невозврата) и аккуратней относиться к выдаче кредитов семьям с 1, 2 детьми (9.2 и 9.5% невозврата, соответственно)
- У неженатые/незамужних максимальный процент невозврата 9.8%. Минимальный процент невозврата у групп "вдовец/вдова" и "в разводе" (6.6 и 7.1%, соответственно). Для молодожен этот показатель ниже (7.5%), чем у клиентов в гражданском браке (9.3%).
- Клиенты со сверхзаработком чаще возвращают кредит (7% невозврата). Однако, люди со средним и высоким заработком отдают кредит реже (9 и 8.6% по невозврату, соответственно) группы с низким доходом (8%). 
- Давать кредит на жилье и недвижимость выгоднее (процент невозврата 6.9 и 7.5%, соответственно), чем на образование и автомобиль (9.2 и 9.4%, соответственно).

**По качеству данных:**

- Количество клиентов из группы многодетных семей значительно меньше остальных групп, при наличии, необходимо дополнить датасет клиентами из этой группы для получения достоверных результатов. 
- Ввести систему проверки входных данных при вводе (например, система не будет позволять вводить отрицательные значения, аномальные значения для параметров)
- Продумать систему проверки при поступлении дублей
- Вводить трудовой стаж в годах, а не в днях
- Продумать новые название столбцов + для зарплаты необходимо указывать в какой валюте она выдается